In [12]:
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML

import tensorflow as tf

In [8]:
wget https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip -O

SyntaxError: invalid syntax (<ipython-input-8-db552d51fecc>, line 1)